# 기존 교재의 예제코드입니다
[CNN-01] 필요한 모듈을 임포트하고 난수의 시드를 설정한다.

In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)


[CNN-02] MNIST 데이터 세트를 준비한다.

In [0]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t


[CNN-03] 첫 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv1 = tf.Variable(tf.truncated_normal([5,5,1,num_filters1],
                                          stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')

b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')


[CNN-04] 두 번째 단계의 합성곱 필터와 풀링 계층을 정의한다.

In [0]:
num_filters2 = 64

W_conv2 = tf.Variable(
            tf.truncated_normal([5,5,num_filters1,num_filters2],
                                stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')


[CNN-05] 전 결합층, 드롭아웃 계층, 소프트맥스 함수를 정의한다.

In [0]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2_drop, w0) + b0)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



[CNN-06] 오차 함수 loss, 트레이닝 알고리즘 train_step, 정답률 accuracy을 정의한다.

In [0]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


[CNN-07] 세션을 준비하고 Variable을 초기화한다.

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


[CNN-08] 파라미터 최적화를 20000회 반복한다.

최종적으로 테스트 세트에 대해 약 99%의 정답률을 얻을 수 있다.

In [0]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = (int)(len(mnist.test.labels) / 4 * c)
            end = (int)(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'cnn_session', global_step=i)

Step: 500, Loss: 1459.561768, Accuracy: 0.954900
Step: 1000, Loss: 955.638245, Accuracy: 0.970900
Step: 1500, Loss: 767.265503, Accuracy: 0.973600
Step: 2000, Loss: 656.851135, Accuracy: 0.979100
Step: 2500, Loss: 580.280396, Accuracy: 0.980200
Step: 3000, Loss: 510.467285, Accuracy: 0.983900
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Step: 3500, Loss: 462.890320, Accuracy: 0.983500
Step: 4000, Loss: 437.105347, Accuracy: 0.985900
Step: 4500, Loss: 399.214722, Accuracy: 0.986800
Step: 5000, Loss: 381.797821, Accuracy: 0.986800
Step: 5500, Loss: 389.958832, Accuracy: 0.985600
Step: 6000, Loss: 377.912903, Accuracy: 0.987000
Step: 6500, Loss: 356.498169, Accuracy: 0.987100
Step: 7000, Loss: 341.254578, Accuracy: 0.988800
Step: 7500, Loss: 355.357300, Accuracy: 0.987800
Step: 8000, Loss: 413.995575, Accuracy: 0.986600
Step: 8500, Loss: 363.434570, Accuracy: 0.987100
Step: 9000, Loss: 297.517395, Accuracy: 0.990000
Step: 9500, Loss: 329.583832, Accu